In [1]:
from batoid_rubin.align_game import AlignGame
import ipywidgets

%matplotlib widget

In [2]:
control_log = ipywidgets.Output()
control_log
# control_log=None

Output()

In [3]:
app = AlignGame(control_log=control_log, nthread=4)
app.display()

Explanation
===========

Use this tool to investigate optical effects of misaligning the Rubin secondary mirror (M2) or camera.  You can
even turn this misalignment investigation into a game by applying hidden random misalignments and using feedback
from both the in-focus PSF images and out-of-focus donut images to try and get the telescope back into alignment.

Layout
------
On the left, you will see a grid of PSFs (in-focus) and donuts (out-of-focus) representing the Rubin focal plane.  
The outer (inner) donuts are intra (extra) focal images of stars obtained by the corner wavefront sensors.  The
grid of in-focus stars sample the center of each Rubin raft.

At the top of the grid, the current wavefront error integrated over both the telescope pupil and field-of-view is
shown.  The number of iterations since the last randomization reset is also shown.

Degrees of freedom
------------------
There are 5 rigid body degrees of freedom for (mis)-aligning both M2 and the camera.  The $dx$ and $dy$ variables
will decenter the optic perpendicular to the optic axis.  The $dz$ variables manipulate the placement of the optic
along the optic axis.  The $Rx$ and $Ry$ variables tilt the optic along the $x$ and $y$ axes.

Degeneracies
------------
Some of the degrees of freedom are (mostly) degenerate.  Try displacing both the camera and M2 in opposite 
directions; you should find that these degrees of freedom mostly cancel each other out.  There are additional
degeneracies present between decentering and rotating optics.

Buttons
-------

The ``Zero`` button will reset the widget to zero misalignment errors.

The ``Randomize`` button will apply a hidden random offset.  You can use this as a start to a game of aligning the 
telescope manually.  Try to manipulate the degrees-of-freedom to bring the telescope back into alignment.

The ``Reveal`` button will show the hidden offsets in the right text panel.

The ``Solve`` button will apply the degrees of freedom that exactly cancel the hidden offsets thus bringing the 
telescope back into alignment.

The ``Control w/ Trunc`` button will apply a control algorithm to the optics in a manner similar to the actual
Rubin active optics.  A forward model fit of the eight donut images is performed to infer the wavefront error 
at each corner.  A precomputed senstivity matrix is then used to determine the commands that should bring the 
telescope back into alignment.  Because of degeneracies (see above) a naive conversion from wavefront errors
to degrees of freedom is ill-formed.  With this button, that degeneracy is addressed by only attempting to
control a non-degenerate subset of degrees of freedom.

The ``Control w/ Penalty`` button performs the same wavefront error estimation as the ``Control w/ Trunc`` button,
but applies a different algorithm to convert wavefront errors into control actions.  In this case, all 10
degrees of freedom are controlled, but a penalty is applied to large commands, preventing the solution from
wandering off along a degeneracy direction.

Note that running either control algorithm will also populate the debug output in the notebook cell above the
main widget showing the donut fits and progress in manipulating degrees of freedom and achieved wavefront error.

In [6]:
import danish
import yaml
from pathlib import Path

In [7]:
Rubin_obsc = yaml.safe_load(open(Path(danish.datadir)/'RubinObsc.yaml'))

In [10]:
danish.DonutFactory(mask_params=Rubin_obsc)